# **Face Attendance System**

Recognizing people by their faces in pictures and video feeds is seen everywhere starting from social media to phone cameras. A face recognition system is built for matching human faces with a digital image. Ultimately what a computer recognizes is pixel values ranging from 0-255. In Computer Vision face recognition has been in since ages and has evolved over the years. Many researchers have come up with many new techniques to efficiently identify and tell apart faces. There are many use cases such as authentication and verification of users.

This practice session covers all the aspects of face recognition based attendance systems. It discusses the challenges faced in face recognition, the face recognition library and building the attendance marking system based on these techniques.

# **Code Implementation**

## **Basic Face Matching** 

First, we get the location of where exactly the face is in the image using face_location() method(which gets the outline of the face) on the RGB image. Then face encodings(markings of eyes, nose, mouth, jaws which remain the same for different images of the same person) are taken using face_encodings() function which returns a list containing 128 measurements. Both these two steps are followed for the original and test image. Then a comparison between these two returned lists is done by the function compare_faces() which returns a list of boolean values(True or False). The face distance function gets the value of that by how much the two images differ. The lower the distance the better the matching and vice versa.

In [ ]:
!python -m pip install pip --upgrade --user -q --no-warn-script-location
!python -m pip install numpy pandas seaborn matplotlib scipy statsmodels sklearn nltk gensim tensorflow keras torch torchvision \
    tqdm scikit-image pillow face_recognition --user -q --no-warn-script-location


import IPython
IPython.Application.instance().kernel.do_shutdown(True)


In [ ]:
# !wget -O andrew_ng.jpg https://wp.technologyreview.com/wp-content/uploads/2017/08/andrew-ng-7.jpg

In [ ]:
# !wget -O ian_godfellow.jpg https://upload.wikimedia.org/wikipedia/commons/thumb/f/fe/Ian_Goodfellow.jpg/330px-Ian_Goodfellow.jpg

In [ ]:
import cv2
import face_recognition as fr
imgAng = fr.load_image_file('andrew_ng.jpg')
Test = fr.load_image_file('ian_godfellow.jpg')
fLoc = fr.face_locations(imgAng)[0]
encodeAng = fr.face_encodings(imgAng)[0]
fLocTest = fr.face_locations(Test)[0]
encTest = fr.face_encodings(Test)[0]
result = fr.compare_faces([encodeAng],encTest)
faceDist = fr.face_distance([encodeAng],encTest)
print(result,faceDist)

## **Building Face Attendance System**

Now we are ready to build a realtime face attendance system wherein webcam captured frames will be matched against the existing database images and if the match is found then it’ll store it in a CSV file called ‘Attendance Register’ along with name and time of capture. Only once the file will store the matched image’s details, if the same image is received again then it’ll not update.

Path setting to the directory containing the image database. Read each image and the images array. Append the filenames into a list called Names and remove the extensio

In [ ]:
# !mkdir ImagesAttendance

In [ ]:
# !wget -O ImagesAttendance/andrew.jpg https://i2.wp.com/syncedreview.com/wp-content/uploads/2018/11/6a7be29fafa971a5952b2e031f63c536463861a6.jpg

In [ ]:
# !wget -O ImagesAttendance/ian.jpg https://i2.wp.com/syncedreview.com/wp-content/uploads/2018/11/6a7be29fafa971a5952b2e031f63c536463861a6.jpg

In [ ]:
# !wget  -O ImagesAttendance/nicole.jpg https://pyxis.nymag.com/v1/imgs/8a4/e6b/b6237f56f0428eedf85e117ef0c669de58-nicole-kidman.rvertical.w330.jpg

In [ ]:
import os
pathlib = 'ImagesAttendance'
images = []
Names = []
myList = os.listdir(pathlib)
print(myList)
for cl in myList:
    currImg = cv2.imread(f'{pathlib}/{cl}')
    images.append(currImg)
    Names.append(os.path.splitext(cl)[0])
print(Names)

Finding face encodings of images in the database and keeping them in a list to use later with incoming frames. 

In [ ]:
def DbEncodings(images):
    encList = []
    for image in images:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        enc = fr.face_encodings(image)[0]
        encList.append(enc)
    return encList

The same process is followed by the first detection face location then getting the face encoding values.

In [ ]:
encodeKnown = DbEncodings(images)
print('Encoding Complete')
 
cap = cv2.VideoCapture(0)

#Iterating through frames 
while True:
    _, img = cap.read()
    image = cv2.resize(img,(0,0),None,0.25,0.25)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    facesInFrame = face_recognition.face_locations(image)
    encodesInFrame = face_recognition.face_encodings(image,facesInFrame)
    #Now the incoming images are tested against the previously-stored encodings. 
    #Then the face distance is also computed. 
    #Lastly, we call the Attendance function along with the person name who is identifie

    for encFace,faceLoc in zip(encodesInFrame,facesInFrame):
        matchList = face_recognition.compare_faces(encodeKnown,encFace)
        faceDis = face_recognition.face_distance(encodeKnown,encFace)
        #print(faceDis)
        match = np.argmin(faceDis)

        if matchList[match]:
            name = Names[match].upper()
            #print(name)
            y,w,z,x = faceLoc
            y, w, z, x1 = y*4,w*4,z*4,x*4
            cv2.rectangle(img,(x,y),(w,z),(0,255,0),2)
            cv2.rectangle(img,(x,z-35),(w,z),(0,255,0),cv2.FILLED)
            cv2.putText(img,name,(x+6,z-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
            Attendance(name)

    cv2.imshow('Webcam',img)
    k = cv2.waitKey(0)
    if k == 27:         # wait for ESC key to exit
        break
        cv2.destroyAllWindows()

Reading from attendance file, Storing data(Name and Time of entry) if previously not stored.

In [ ]:
def Attendance(name):
    with open('Attendance_Register.csv','r+') as f:
        DataList = f.readlines()
        names = []
        for data in DataList:
            ent = data.split(',')
            names.append(ent[0])
        if name not in names:
            curr = datetime.now()
            dt = curr.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dt}')

Capturing video frames

#**Related Articles:**

> * [Face Attendance System](https://analyticsindiamag.com/a-complete-guide-on-building-a-face-attendance-system/)

> * [6 MNIST Image Dataset](https://analyticsindiamag.com/mnist/)

> * [Vision Transformers](https://analyticsindiamag.com/hands-on-vision-transformers-with-pytorch/)

> * [MONAI](https://analyticsindiamag.com/monai-datatsets-managers/)

> * [Guide to Pystiche](https://analyticsindiamag.com/pystiche/)

> * [MODNet](https://analyticsindiamag.com/modnet-a-trimap-free-realtime-portrait-matting/)

> * [Monk's AI](https://analyticsindiamag.com/build-computer-vision-applications-with-few-lines-of-code-using-monk-ai/)